In [56]:
from langgraph.graph import StateGraph ,START,END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from typing import TypedDict
import os

In [57]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [58]:
model = ChatOpenAI(api_key=api_key,model="gpt-4o")

In [59]:
class BlogState(TypedDict):
    title:str
    outline:str
    blog:str
    score:int

In [60]:
def create_outline(state:BlogState)-> BlogState:
    title = state['title'] 
    prompt = f"Draft me the detial outline on {title}.Use professional + Precise wording"
    new_outline = model.invoke(prompt).content

    state['outline'] = new_outline

    return state

In [61]:
def create_blog(state:BlogState)-> BlogState:
    outline = state['outline']

    prompt = f'Draft me good and informative + Precise blog on {outline}. Use emojies to make it interative and use friendly , fun and professional way'

    blog = model.invoke(prompt).content

    state['blog'] = blog

    return state

In [64]:
def evaluate(state:BlogState)-> BlogState:
    outline = state['outline']
    blog = state['blog']

    prompt = f'Analyze the outline -> {outline} \n and blog -> {blog} \n.Give total score out of 10'

    score = model.invoke(prompt).content

    state['score'] = score

    return state

In [ ]:
# create graph

graph = StateGraph(BlogState)

## ADD NODE

graph.add_node("create_outline",create_outline)
graph.add_node("create_blog",create_blog)
graph.add_node("evaluate",evaluate)

## Add Edge

graph.add_edge(START,"create_outline")
graph.add_edge("create_outline","create_blog")
graph.add_edge("create_blog","evaluate")
graph.add_edge("evaluate",END)

## create workflow 

workflow = graph.compile()

TypeError: StateGraph.add_edge() takes 3 positional arguments but 4 were given

In [ ]:
inital_state = {"title":"How can i full stack software developer with MERN stack can become a AI Engineer using langchain and langgraph and build exiting projects?"}

final_result = workflow.invoke(inital_state)

print(final_result)

In [ ]:
from IPython.display import Image

Image(workflow.get_graph().draw_mermaid_png())

In [ ]:
print("--BLOG Outline--")
print(final_result['outline'])



In [ ]:
print("--BLOG--")
print(final_result['blog'])